БД: `PostgreSQL` \
Описание таблиц БД `test_data`:

1. Таблица `CUSTOMER_OTP` - cодержит информацию о клиентах. Столбцы:
- `CUST_ID` (INT): Уникальный идентификатор клиента.
- `FULLNAME` (VARCHAR(255)): фамилия клиента.
- `SEX` (CHAR(1)): Пол клиента ('М' - мужской, 'Ж' - женский).
- `BIRTH_DT` (DATE): Дата рождения клиента.
- `VALID_FROM` (DATE): Дата начала действия записи о клиенте.
- `VALID_TO` (DATE): Дата окончания действия записи о клиенте.
 
2. Таблица `CUSTOMER_CONTACT_OTP` - cодержит контактные данные клиентов. Cтолбцы:
- `CUST_ID` (INT): Уникальный идентификатор клиента (связь с таблицей CUSTOMER_OTP).
- `TYPE` (VARCHAR(255)): Тип контактной информации ('EMAIL', 'PHONE', 'ADDRESS' и т.д.).
- `VALUE` (VARCHAR(255)): Значение контактной информации (например, email, номер телефона или адрес).
- `VALID_FROM` (DATE): Дата начала действия записи о контакте.
- `VALID_TO` (DATE): Дата окончания действия записи о контакте.
 
3. Таблица `TRANSACTION_OTP` - cодержит информацию о транзакциях клиентов. Столбцы:
- `CUST_ID` (INT): Уникальный идентификатор клиента (связь с таблицей CUSTOMER_OTP).
- `TRANSACTION_DT` (DATE): Дата транзакции.
- `CURRENCY_ID` (VARCHAR(3)): Код валюты транзакции ('RUR' - российский рубль, 'USD' - доллар США, 'EUR' - евро и т.д.).
- `TRANSACTION_AMT` (DECIMAL(10, 2)): Сумма транзакции.

4. Таблица `CURRENCY_RATE_OTP` - cодержит информацию о курсах валют. Столбцы:
- `CURRENCY_ID` (VARCHAR(3)): Код валюты ('RUR', 'USD', 'EUR' и т.д.).
- `EXCH_RATE` (DECIMAL(10, 2)): Курс обмена для соответствующей валюты.
- `VALID_FROM` (DATE): Дата начала действия курса.
- `VALID_TO` (DATE): Дата окончания действия курса.

### Задача 1
Выберите клиентов пенсионного возраста: женщин >= 60, мужчин >= 65. Выборку сделайте на день формирования запроса.

Также, ограничьте выборку только теми клиентами, данные которых актуальны на текущий момент (где столбец valid_to не является уже прошедшей датой).

Столбцы в результате: \
`cust_id`, \
`fullname`, \
`sex`, \
`birth_dt`

### Решение:
```
SELECT 
    cust_id, 
    fullname,
    sex,
    birth_dt
FROM 
    test_data.CUSTOMER_OTP 
WHERE 
    CASE 
     	WHEN sex = 'Ж' THEN birth_dt + INTERVAL '60 years'
     	ELSE birth_dt + INTERVAL '65 years'
    END <= NOW() 
AND valid_to = '4000-01-01';
```

### Задача 2
Выбрать текущие ФИО клиента, телефон, e-mail, адрес.

Столбцы в результате: \
`cust_id`, \
`fullname`, \
`phone`, \
`email`, \
`address`

### Решение:
```
SELECT 
    co.cust_id, 
    co.fullname, 
    MIN(CASE WHEN cco.type = 'PHONE' THEN cco.value END) as PHONE,
    MIN(CASE WHEN cco.type = 'EMAIL' THEN cco.value END) as EMAIL,
    MIN(CASE WHEN cco.type = 'ADDRESS' THEN cco.value END) as ADDRESS
FROM 
    test_data.CUSTOMER_CONTACT_OTP as cco
INNER JOIN 
    test_data.CUSTOMER_OTP as co ON cco.cust_id = co.cust_id AND co.valid_to = '4000-01-01'
WHERE 
    cco.valid_to = '4000-01-01'
GROUP BY
    co.cust_id, 
    co.fullname
```

### Задача 3
Посчитать сумму транзакций клиента в рублях за январь 2021 года.

Столбцы в результате: \
`cust_id`, \
`fullname`, \
`amount_rur`

### Решение:
```
SELECT t.cust_id, 
       c.fullname,
       SUM(t.transaction_amt * cr.exch_rate) AS amount_rur
FROM
    test_data.TRANSACTION_OTP AS t
JOIN 
    test_data.CURRENCY_RATE_OTP AS cr ON t.currency_id = cr.currency_id
    AND cr.valid_to >= t.transaction_dt
    AND cr.valid_from <= t.transaction_dt 
JOIN 
    test_data.CUSTOMER_OTP AS c ON c.cust_id = t.cust_id
WHERE 
    date_trunc('MONTH',  transaction_dt) = '2021-01-01'
GROUP BY
    t.cust_id, 
    c.fullname
```

### Задача 4
Выбрать только уникальных по ФИО клиентов.

Примечание: ФИО должны быть актуальны на текущий момент и включать однофамильцев. \
То есть, Иванов с cust_id = 1 и Иванов c cust_id = 10 - разные люди и оба должны попасть в выборку

Столбцы в результате: \
`fullname`

### Решение:
```
SELECT 
    fullname
FROM 
    test_data.CUSTOMER_OTP 
WHERE 
    cust_id IN 
        (SELECT
            distinct cust_id 
        FROM
            test_data.CUSTOMER_OTP)
AND valid_to = '4000-01-01'
```

### Задача 5
Выберите текущую и предыдущую фамилию клиентов, которые меняли свои фамилии. 

Столбцы в результате: \
`cust_id`, \
`fullname`, \
`valid_from`, \
`previous_fullname`

### Решение:
```
WITH old_clients AS (
SELECT 
    cust_id,
    fullname, 
    LAG (fullname) OVER(PARTITION BY cust_id ORDER BY valid_from ROWS between unbounded preceding AND UNBOUNDED following) as previous_fullname,    
    valid_from,
    valid_to    
FROM 
    test_data.CUSTOMER_OTP
)
SELECT   
    cust_id, 
    fullname, 
    valid_from,
    previous_fullname
FROM
    old_clients
WHERE
    valid_to = '4000-01-01';
```